In [ ]:

import argparse
import copy
import os
from enum import Enum
from typing import Dict, List, Union

import numpy as np
import pandas as pd
import torch
from config.configuration import Config
from data.dataloader import GeneralTrainerDataLoader
from data.dataset import GeneralDataset
from data.interaction import Interaction
from data.utils import data_reparation
from models import NeuMF
from root import DATASET_DIR, ROOT_DIR, absolute
from torch.nn.utils import rnn as rnn_utils
from torch.utils.data import Dataset as TorchDataset
from trainer import Trainer
from utils.logger import init_logger

### TODO
- 添加验证集
- NGCF
- LightGCN
- 所有代码整体过一遍
- tensorboard
- 训练参数整理
- checkpoint逻辑
- 日志 ✅

In [ ]:

parser = argparse.ArgumentParser()
parser.add_argument(
    "--dataset", "-d", type=str, default="wsdream-rt", help="name of datasets"
)

args, _ = parser.parse_known_args()

config = Config(model="NeuMF", dataset=args.dataset)


init_logger(config)

dataset = GeneralDataset(config)
train_data, test_data = data_reparation(config, dataset)
model = NeuMF(config, dataset)
trainer = Trainer(config, model)

trainer.fit(train_data, test_data, saved=False, show_progress=True)


In [1]:
import numpy as np

# 生成0到99的数字数组
data = np.arange(100)

# 随机采样20%的数据
sampled_data = np.random.choice(data, size=int(len(data) * 0.2), replace=False)

# 获取剩下的数据
remaining_data = np.setdiff1d(data, sampled_data)

print("采样数据:")
print(sampled_data)

print("剩下的数据:")
print(remaining_data)

采样数据:
[55 88  5 74 71 98  3 46 30 92 48 24 40 64  4 21 54 76 16 22]
剩下的数据:
[ 0  1  2  6  7  8  9 10 11 12 13 14 15 17 18 19 20 23 25 26 27 28 29 31
 32 33 34 35 36 37 38 39 41 42 43 44 45 47 49 50 51 52 53 56 57 58 59 60
 61 62 63 65 66 67 68 69 70 72 73 75 77 78 79 80 81 82 83 84 85 86 87 89
 90 91 93 94 95 96 97 99]
